In [1]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
n_gpu_layers = 5  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [21]:
_PATH_LLAMA_ = "/Users/niteshkumarsharma/.cache/lm-studio/models/TheBloke/Llama-2-7b-Chat-GGUF/llama-2-7b-chat.Q5_K_M.gguf"
_PATH_MISTRAL_ = "/Users/niteshkumarsharma/.cache/lm-studio/models/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q5_K_M.gguf"

In [26]:
MISTRAL = LlamaCpp(model_path=_PATH_MISTRAL_,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=4096,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/niteshkumarsharma/.cache/lm-studio/models/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q5_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q5_K     [  4096, 14336,     1,     1 ]
llama_mode

In [25]:
LLAMA = LlamaCpp(
    model_path=_PATH_LLAMA_,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=4096,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/niteshkumarsharma/.cache/lm-studio/models/TheBloke/Llama-2-7b-Chat-GGUF/llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor

In [5]:
from langchain.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain

In [6]:
loader = WebBaseLoader("https://www.espn.com/")

In [7]:
data = loader.load()

In [8]:
data

[Document(page_content="\n\n\n\n\n\n\n\n\nESPN - Serving Sports Fans. Anytime. Anywhere.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        Skip to main content\n    \n\n        Skip to navigation\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<\n\n>\n\n\n\n\n\n\n\n\n\nMenuESPN\n\n\nSearch\n\n\n\nscores\n\n\n\nNFLMLBNCAAFNBANHLSoccer…NCAAMNCAAWSports BettingBoxingCFLNCAACricketF1GolfHorseLLWSMMANASCARNBA G LeagueOlympic SportsPLLRacingRN BBRN FBRugbyTennisWNBAWWEX GamesXFLMore ESPNFantasyListenWatchESPN+\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\nSUBSCRIBE NOW\n\n\n\n\n\nNFL PrimeTime\n\n\n\n\n\n\n\nLALIGA\n\n\n\n\n\n\n\nNHL Preseason: Select Games\n\n\n\n\n\n\n\nThe Pat McAfee Show\n\n\n\n\n\n\n\nICC Cricket World Cup: Starts Thursday\n\n\n\n\n\n\n\nNFL Week 4 Overreactions\n\n\nQuick Links\n\n\n\n\nMLB Playoffs\n\n\n\n\n\n\n

In [9]:
chain = load_summarize_chain(llm, chain_type="stuff")

In [10]:
chain(data)

 SUNDAY SUNMATIHS 59ers NFL Nation reporter Solomon Wilcots was caught off guard by the massive shake-up in the NFC East, as the Eagles and Cowboys made significant moves to upgrade their rosters. The Giants and Redskins also made some notable additions, but are they enough to keep up with Philly and Dallas? Plus, what's next for the Patriots after their major trade deadline deal for former Pro Bowl QB Matthew Stafford? And can anyone stop the Chiefs in the AFC West? ESPN's NFL Nation reporters break down the latest moves around the league and provide insights into what they mean for each team's chances in 2023.
































































ESPN ESPN ESPNP ESPNESPN ESPN ESPN ESPN ESPN ESPN ESPN ESPN ESPN ESP


llama_print_timings:        load time =  6405.35 ms
llama_print_timings:      sample time =   251.53 ms /   256 runs   (    0.98 ms per token,  1017.78 tokens per second)
llama_print_timings: prompt eval time = 18000.67 ms /  2914 tokens (    6.18 ms per token,   161.88 tokens per second)
llama_print_timings:        eval time = 13307.49 ms /   255 runs   (   52.19 ms per token,    19.16 tokens per second)
llama_print_timings:       total time = 32145.53 ms


{'input_documents': [Document(page_content="\n\n\n\n\n\n\n\n\nESPN - Serving Sports Fans. Anytime. Anywhere.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        Skip to main content\n    \n\n        Skip to navigation\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<\n\n>\n\n\n\n\n\n\n\n\n\nMenuESPN\n\n\nSearch\n\n\n\nscores\n\n\n\nNFLMLBNCAAFNBANHLSoccer…NCAAMNCAAWSports BettingBoxingCFLNCAACricketF1GolfHorseLLWSMMANASCARNBA G LeagueOlympic SportsPLLRacingRN BBRN FBRugbyTennisWNBAWWEX GamesXFLMore ESPNFantasyListenWatchESPN+\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\nSUBSCRIBE NOW\n\n\n\n\n\nNFL PrimeTime\n\n\n\n\n\n\n\nLALIGA\n\n\n\n\n\n\n\nNHL Preseason: Select Games\n\n\n\n\n\n\n\nThe Pat McAfee Show\n\n\n\n\n\n\n\nICC Cricket World Cup: Starts Thursday\n\n\n\n\n\n\n\nNFL Week 4 Overreactions\n\n\nQuick Links\n\n\n\n\nMLB Pl

In [11]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate


from langchain.chains.combine_documents.stuff import StuffDocumentsChain

In [12]:
## Prompt Defination
skills_template = """Get Key Skills from Job Description Below:
"{job_desc}"
KEY SKILLS:"""

skill_prompt = PromptTemplate.from_template(skills_template)

In [13]:
## Prompt Defination
prompt_summary_template = """CREATE NEW RESUME SUMMARY BY MODIFYING BELOW SUMMARY:
"{summary}"

BY UTILISING JOB DESCRIPTION PROVIDED BELOW:
"{job_desc}

DO NOT EXCEED WORD COUNT BY: 
"{limit}

NEW RESUME SUMMARY:"""

prompt_summary = PromptTemplate.from_template(prompt_summary_template)

In [14]:
PROFILE_SUMMARY = """
Hello, I am Nitesh, an Experienced Machine Learning professional with 4+ years of industry expertise, specialising in leveraging advanced statistical & predictive modelling, machine learning and data analysis to drive results with digital transformation. 
Backed with Research experience as a Master’s from IIT Madras, I have a solid understanding of statistics, machine learning, signal processing & Time series analysis
I have tackled diverse challenges, from designing experiments, building and implementing research papers, and operating models at scale. My proudest accomplishment is designing & developing a scalable and cost-efficient B2B product for customers that can handle close to 10 TBs of sensor data utilising AWS Services and Airflow
"""

In [38]:
jd_description = """
E2E Data & Analytics team at Walmart is focused on using the latest and greatest in time series forecasting, machine learning and statistics to solve business forecasting problems within supply chain process flow. We build time series forecasting models and machine learning pipelines in a big data ecosystem to empower decision-making. Advanced analytical and forecasting algorithms driven by our team help Walmart to optimize supply chain at operational, tactical and strategic levels, business practices, reduce cost and pass those benefits to our customers.


You will be contributing towards building forecasting/prediction/root-cause algorithms by closely collaborating with our stakeholders at Bentonville and provide solution to business challenges. In most cases the prioritization of business challenges is based on impact and hence your contributions can have direct impact to our businesses.


What you'll do:


As a Senior Data Scientists for Walmart, you’ll have the opportunity to


Play a key role to solve complex time series forecasting problems, pivotal to Walmart’s business and drive actionable insights from terabytes of data


Leverage data science tools and techniques, keeping abreast with the latest in the community to solve problems for Walmart.


Develop PoC, present lucidly to the business and evolve the solutions


Take forward the solutions into Pipelines/APIs as needed by the business


Research, learn/disseminate & adapt new technologies to solve problems & improve upon existing solutions


Brainstorm, review and mentor junior associates in providing robust data science solutions


What you'll bring:


    B.Tech or MSc with 6+ years or M.Tech with 4+ years of relevant experience in AI time series forecasting and machine learning / artificial intelligence
    Problem solver at heart with experience in Time Series Forecasting, Machine learning and Data Science
    Proficiency with Python/PySpark/BiqQuery etc
    Proficiency with ML libraries sklearn/pytorch/tensorflow etc 
    Good knowledge of commonly used design patterns in data science
    Cloud friendliness (GCP/Azure etc) to leverage distributed computing/scalability
    Good communication/influencing/mentoring skills with inclination to high ownership and commitment
"""

In [39]:
summary_prompt = prompt_summary.format(summary=PROFILE_SUMMARY, job_desc=jd_description, limit=100)
skills_prompt = skill_prompt.format(job_desc=jd_description)

In [40]:
skills_prompt

'Get Key Skills from Job Description Below:\n"\nE2E Data & Analytics team at Walmart is focused on using the latest and greatest in time series forecasting, machine learning and statistics to solve business forecasting problems within supply chain process flow. We build time series forecasting models and machine learning pipelines in a big data ecosystem to empower decision-making. Advanced analytical and forecasting algorithms driven by our team help Walmart to optimize supply chain at operational, tactical and strategic levels, business practices, reduce cost and pass those benefits to our customers.\n\n\nYou will be contributing towards building forecasting/prediction/root-cause algorithms by closely collaborating with our stakeholders at Bentonville and provide solution to business challenges. In most cases the prioritization of business challenges is based on impact and hence your contributions can have direct impact to our businesses.\n\n\nWhat you\'ll do:\n\n\nAs a Senior Data S

In [41]:
LLAMA(skills_prompt)

Llama.generate: prefix-match hit




* Time series forecasting
* Machine learning
* Data science
* Predictive analytics
* Statistics
* Algorithm development
* Data visualization
* Cloud computing
* Distributed computing
* Python/PySpark/BiqQuery etc
* ML libraries sklearn/pytorch/tensorflow etc
* Design patterns in data science
* Communication/influencing/mentoring skills with high ownership and commitment.


































































































































































'\n\n* Time series forecasting\n* Machine learning\n* Data science\n* Predictive analytics\n* Statistics\n* Algorithm development\n* Data visualization\n* Cloud computing\n* Distributed computing\n* Python/PySpark/BiqQuery etc\n* ML libraries sklearn/pytorch/tensorflow etc\n* Design patterns in data science\n* Communication/influencing/mentoring skills with high ownership and commitment.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

llama_print_timings:        load time =  2830.00 ms
llama_print_timings:      sample time =   232.32 ms /   256 runs   (    0.91 ms per token,  1101.94 tokens per second)
llama_print_timings: prompt eval time =  7430.44 ms /   545 tokens (   13.63 ms per token,    73.35 tokens per second)
llama_print_timings:        eval time = 10331.34 ms /   255 runs   (   40.52 ms per token,    24.68 tokens per second)
llama_print_timings:       total time = 18514.72 ms


In [42]:
MISTRAL(skills_prompt)

Llama.generate: prefix-match hit


 Time Series Forecasting, Machine Learning, Data Science, Python, PySpark, BigQuery, SQL, Cloud Computing, Data Visualization, Communication Skills, Problem Solving Skills, Mentoring, Adaptability


llama_print_timings:        load time =  7047.55 ms
llama_print_timings:      sample time =    34.99 ms /    50 runs   (    0.70 ms per token,  1428.78 tokens per second)
llama_print_timings: prompt eval time =  7413.94 ms /   523 tokens (   14.18 ms per token,    70.54 tokens per second)
llama_print_timings:        eval time =  2050.93 ms /    49 runs   (   41.86 ms per token,    23.89 tokens per second)
llama_print_timings:       total time =  9576.15 ms


' Time Series Forecasting, Machine Learning, Data Science, Python, PySpark, BigQuery, SQL, Cloud Computing, Data Visualization, Communication Skills, Problem Solving Skills, Mentoring, Adaptability'

In [46]:
LLAMA(summary_prompt)

Llama.generate: prefix-match hit



As a seasoned Machine Learning professional with over 4 years of experience in driving digital transformation through advanced statistical & predictive modeling, I specialize in leveraging AWS Services and Airflow to design scalable and cost-efficient solutions for diverse challenges. Backed by my Research experience from IIT Madras, I possess a solid understanding of statistics, machine learning, signal processing, and time series analysis. My proudest accomplishment is developing a B2B product that can handle close to 10 TBs of sensor data. As a Senior Data Scientist at Walmart, I contribute towards solving complex time series forecasting problems, driving actionable insights from terabytes of data, and evolving solutions into Pipelines/APIs. I possess excellent problem-solving skills, proficiency with Python/PySpark/BigQuery, and good communication/influencing/mentoring skills with a cloud-friendly approach.


llama_print_timings:        load time =  2830.00 ms
llama_print_timings:      sample time =   205.80 ms /   199 runs   (    1.03 ms per token,   966.96 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 13113.92 ms /   199 runs   (   65.90 ms per token,    15.17 tokens per second)
llama_print_timings:       total time = 13790.98 ms


'\nAs a seasoned Machine Learning professional with over 4 years of experience in driving digital transformation through advanced statistical & predictive modeling, I specialize in leveraging AWS Services and Airflow to design scalable and cost-efficient solutions for diverse challenges. Backed by my Research experience from IIT Madras, I possess a solid understanding of statistics, machine learning, signal processing, and time series analysis. My proudest accomplishment is developing a B2B product that can handle close to 10 TBs of sensor data. As a Senior Data Scientist at Walmart, I contribute towards solving complex time series forecasting problems, driving actionable insights from terabytes of data, and evolving solutions into Pipelines/APIs. I possess excellent problem-solving skills, proficiency with Python/PySpark/BigQuery, and good communication/influencing/mentoring skills with a cloud-friendly approach.'

In [ ]:
## LLAMA 2 7B Resume Profile Summary based on Default Profile Summary &  Provied JD ##
"""Nitesh is a seasoned Machine Learning professional with over 4 years of industry experience in leveraging advanced statistical & predictive modeling, machine learning and data analysis 
to drive results through digital transformation. Backed by his Master's degree from IIT Madras, he has a solid understanding of statistics, machine learning, signal processing & Time series analysis. 
He has tackled diverse challenges, from designing experiments, building and implementing research papers, and operating models at scale. 

His proudest accomplishment is designing and developing a scalable and cost-efficient B2B product for customers that can handle close to 10 TBs of sensor data utilizing AWS Services and Airflow.
With his passion for staying up-to-date with recent advancements in NLP and large language models, he is well-versed in implementing end-to-end machine learning pipelines, conducting experiments and 
benchmarking to assess the performance of various model architectures and optimizing hyperparameters. He has experience in deploying AI applications powered by complex deep learning models in 
the field of NLP and is proficient in Python, Java, LP, Machine Learning, and Deep Learning (TensorFlow and Pytorch"""


In [ ]:
## Mistral Instruct Profile Summary based on Default Profile Summary &  Provied JD ##

"""Hello, I am Nitesh, an experienced AI/ ML Sr Consultant with a passion for driving digital transformation through advanced statistical & predictive modelling and machine learning. 
   Backed by 8+ years of industry experience in web analytics and related fields, my expertise includes deep dive into emerging product roadmap, trending customer problems, and 
   deriving referential deliverables to implementation teams. I have hands-on experience working with large language models such as GPT, LLAMA, BERT, or 
   Transformer-based architectures, conducting experiments and benchmarking to assess model performance and optimize hyperparameters, and troubleshooting 
   issues during training and deployment. 
   My strong technical knowledge of predictive modeling, feature engineering, model evaluation, and selection has enabled me to deploy AI 
   applications powered by complex deep learning models in the field of NLP, and I am proficient in Python, Java, LP, Machine Learning, and Deep Learning (TensorFlow and Pytorch). 
   In addition to my technical expertise, I have a solid understanding of production machine learning workflow, an ongoing understanding of current state-of-the-art NLU architectures, 
   methods, and processes, and experience in fine tuning and customizing LLM."""


In [45]:
MISTRAL(summary_prompt)

Llama.generate: prefix-match hit



"
Hello, I am Nitesh, an Experienced Machine Learning professional with 6+ years of industry expertise in AI time series forecasting and machine learning. With a Bachelor's degree in Computer Science from [X University], I have a solid understanding of statistics, machine learning, signal processing & Time series analysis. 
I am a problem solver at heart, with experience in developing and implementing robust data science solutions that drive results with digital transformation. My proficiency in Python/PySpark/BiqQuery, along with my expertise in ML libraries such as sklearn, pytorch, and tensorflow has equipped me to tackle diverse challenges in the field of time series forecasting and machine learning. Additionally, I am cloud-friendly and have experience working with distributed computing and scalability on GCP/Azure platforms. My excellent communication, influencing, and mentoring skills, coupled with my inclination towards high ownership and commitment, make me a valuable asset t


llama_print_timings:        load time =  7047.55 ms
llama_print_timings:      sample time =   221.81 ms /   221 runs   (    1.00 ms per token,   996.35 tokens per second)
llama_print_timings: prompt eval time =  7769.95 ms /   733 tokens (   10.60 ms per token,    94.34 tokens per second)
llama_print_timings:        eval time =  9606.93 ms /   220 runs   (   43.67 ms per token,    22.90 tokens per second)
llama_print_timings:       total time = 18104.08 ms


'\n"\nHello, I am Nitesh, an Experienced Machine Learning professional with 6+ years of industry expertise in AI time series forecasting and machine learning. With a Bachelor\'s degree in Computer Science from [X University], I have a solid understanding of statistics, machine learning, signal processing & Time series analysis. \nI am a problem solver at heart, with experience in developing and implementing robust data science solutions that drive results with digital transformation. My proficiency in Python/PySpark/BiqQuery, along with my expertise in ML libraries such as sklearn, pytorch, and tensorflow has equipped me to tackle diverse challenges in the field of time series forecasting and machine learning. Additionally, I am cloud-friendly and have experience working with distributed computing and scalability on GCP/Azure platforms. My excellent communication, influencing, and mentoring skills, coupled with my inclination towards high ownership and commitment, make me a valuable as

In [ ]:
llm(skills_prompt)

In [66]:
## Defining StuffDocumentsChain
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text"
)

In [67]:
docs = loader.load()

In [68]:
print(stuff_chain.run(docs))

Llama.generate: prefix-match hit




Please provide one worded key skill that you see in the job description above.

Please provide one worded key skill that you see in the job description above.



llama_print_timings:        load time =   936.89 ms
llama_print_timings:      sample time =    14.47 ms /    19 runs   (    0.76 ms per token,  1312.97 tokens per second)
llama_print_timings: prompt eval time =  2649.11 ms /    29 tokens (   91.35 ms per token,    10.95 tokens per second)
llama_print_timings:        eval time =  1206.90 ms /    18 runs   (   67.05 ms per token,    14.91 tokens per second)
llama_print_timings:       total time =  3925.97 ms


In [17]:
import requests

In [22]:
URL = "https://www.linkedin.com/jobs/collections/recommended/?currentJobId=3714806549"
page = requests.get(URL)

In [32]:
URL = "https://www.linkedin.com/jobs/collections/recommended/?currentJobId=3714806549"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")


In [34]:
results = soup.find(div="job-details-jobs-unified-top-card__content--two-pane")

In [35]:
results